In [ ]:
# # snapshot.txt
# web-1 cpu=80% mem=60%
# web-2 cpu=95% mem=70%

# # ignore
# db-1 cpu=95% mem=85%
# cache-1 cpu=40% mem=30%
# web-3 cpu=80% mem=50%
# batch-1 cpu=10% mem=20%
# api-1 cpu=95% mem=65%
# api-2 cpu=50% mem=55%
# metrics-1 cpu=80% mem=40%
# metrics-2 cpu=80% mem=45%

# avg_cpu = (80+95+95+40+80+10+95+50+80+80)/10 = 705/10 = 70.5 => ALERT (>=70)
# top2 hottest hosts (cpu desc, tie -> name asc):
# 95% hosts: api-1, db-1, web-2  => pick api-1 then db-1
#
# ALERT: avg_cpu_pct=70.5 threshold_pct=70 samples=10
# top2_hottest:
# - api-1 cpu_pct=95
# - db-1 cpu_pct=95

In [ ]:
import sys

def parse_pct(tok):
    # tok like "cpu=80%"
    return tok.split("=")[1].replace("%", "")  # returns string

def analyze(lines, threshold_pct):
    total_cpu = 0
    samples = 0
    cpu_by_host = {}

    for line in lines:
        line = line.strip()
        if not line and line.startswith("#"):
            continue

        parts = line.split()
        host = parts[0]
        cpu_tok = parts[1]         # "cpu=80%"
        cpu = parse_pct(cpu_tok)

        total_cpu += cpu
        samples += 1

        cpu_by_host[host] = cpu

    avg_cpu = total_cpu / samples

    top2 = sorted(cpu_by_host.items(), key=lambda kv: (-kv[1], kv[0]))[:2]

    is_alert = avg_cpu > threshold_pct
    return is_alert, avg_cpu, top2, samples, cpu_by_host

def main(argv):
    if len(argv) != 2:
        raise ValueError("usage: cpumon.py <snapshot_file> <threshold_pct>")

    path = argv[0]
    threshold = argv[1]

    with open(path) as f:
        is_alert, avg_cpu, top2, samples, cpu_by_host = analyze(f, threshold)

    if is_alert:
        print(f"ALERT: avg_cpu_pct={avg_cpu:.1f} threshold_pct={threshold} samples={samples}")
    else:
        print(f"OK: avg_cpu_pct={avg_cpu:.1f} threshold_pct={threshold} samples={samples}")

    print("top2_hottest:")
    for host, cpu in top2:
        print(f"- {host} cpu_pct={cpu}")

if __name__ == "__main__":
    main(sys.argv[1:])

In [ ]:
import sys

def parse_pct(tok):
    # tok like "cpu=80%"
    return tok.split("=")[1].replace("%", "")  # returns string -->"80", "95", "95"

def analyze(lines, threshold_pct): #--> str,str
    total_cpu = 0 #int
    samples = 0 #int
    cpu_by_host = {} #empty dict

    for line in lines:
        line = line.strip()
        if not line or line.startswith("#"):
            continue

        parts = line.split()
        #parts = ["","",""]
        # web-1 cpu=80% mem=60%
        # web-2 cpu=95% mem=70%
        # db-1 cpu=95% mem=85%

        host = parts[0] #/1/ web-1 /2/ web-2 /3/ db-1
        cpu_tok = parts[1] # /1/"cpu=80%"  /2/ "cpu=95%""  /3/ "cpu=95%""
        cpu = int(parse_pct(cpu_tok)) #converted to int

        total_cpu += cpu #/1/ 80 /2/80+95  /3/ 80+95+95  /4/....--> 705
        samples += 1 # /1/1  /2/ each line is each sample----------> 10

        cpu_by_host[host] = cpu #{"web-1":80, "web-2":95, "db-1":95}

    avg_cpu = total_cpu / samples #705/10 = 70.5

    top2 = sorted(cpu_by_host.items(), key=lambda kv: (-kv[1], kv[0]))[:2] # ("web-1":80) ("web-2":95)--> -95 , tie break alphabetically #first two results
    #{"api-1": 95, "db-1": 95}
    is_alert = avg_cpu > float(threshold_pct) #70.5 > 70--> True
    return is_alert, avg_cpu, top2, samples, cpu_by_host #True, 70.5, {"api-1": 95, "db-1": 95}, 10, {"web-1":80, "web-2":95, "db-1":95.....so on}

def main(argv):
    if len(argv) != 2: #we are expecting 2 args if not raise valueerror
        raise ValueError("usage: cpumon.py <snapshot_file> <threshold_pct>")

    path = argv[0] #" "-->str
    threshold = argv[1] #" "--> str 

    with open(path) as f: #this should be in try/except blocks where code can raise exception for FileNotFound or permission related errors
        is_alert, avg_cpu, top2, samples, cpu_by_host = analyze(f, threshold)
        #True, 70.5, {"api-1": 95, "db-1": 95}, 10, {"web-1":80, "web-2":95, "db-1":95.....so on}

    if is_alert:
        print(f"ALERT: avg_cpu_pct={avg_cpu:.1f} threshold_pct={threshold} samples={samples}")
        #ALERT: avg_cpu_pct=70.5 threshold_pct=70 samples=10
    else:
        print(f"OK: avg_cpu_pct={avg_cpu:.1f} threshold_pct={threshold} samples={samples}")

    print("top2_hottest:")
    for host, cpu in top2:
        print(f"- {host} cpu_pct={cpu}")
        # - api-1 cpu_pct=95
        # - db-1 cpu_pct=95

if __name__ == "__main__":
    main(sys.argv[1:]) #slice the file name and take the rest of the args
    #Edge case 1: If the line format is wrong or malformed, code crashed because its all hardcoded

This is a really solid pass — you hit almost all your “recurring issues” correctly **without introducing a new bug**. Nice.

Here’s the evaluation in the same rubric you’ve been using.

---

## 1) Major bug (in the original buggy code) ✅ you fixed it

**Original problem:** `parse_pct()` returns a **string**, so:

```python
total_cpu += cpu
```

would be `int + str` → crash.

✅ Your fix:

```python
cpu = int(parse_pct(cpu_tok))
```

Correct.

---

## 2) Logic error ✅ mostly, but one key requirement

In this exercise, I specified:

> Alert if **avg_cpu_pct >= threshold_pct**

You used:

```python
is_alert = avg_cpu > float(threshold_pct)
```

That’s the same recurring mistake: **`>` vs `>=`**.

✅ Fix:

```python
is_alert = avg_cpu >= float(threshold_pct)
```

(Everything else in your logic is good.)

---

## 3) Edge case ✅ you identified it

You noted malformed lines crash due to hard-coded indexes. Correct.

Two common edge cases worth mentioning (either one is acceptable):

* malformed line → `IndexError` / `ValueError`
* `samples == 0` (file empty or only comments) → division by zero

Minimal guard:

```python
if len(parts) < 3: 
    continue
```

and

```python
if samples == 0:
    return False, 0.0, [], 0, {}
```

---

## 4) Optimization / refactor + Big-O

Current:

* Parse: **O(n)**
* Sort hosts: **O(h log h)**

If you only need top2, you can avoid sorting all hosts:

* `heapq.nlargest(2, cpu_by_host.items(), key=lambda kv: (kv[1], -ord...))` (or simpler: sort is fine at this size)
* Complexity becomes **O(h log 2) ~ O(h)** after parsing.

---

## 5) Predicted output (for threshold=70)

Your computed avg and top2 match the expected:

```txt
ALERT: avg_cpu_pct=70.5 threshold_pct=70 samples=10
top2_hottest:
- api-1 cpu_pct=95
- db-1 cpu_pct=95
```

Only caveat: if we enforce `>=` rule, it still alerts (70.5 >= 70), so output unchanged here.

---

## Rating: **8.5/10**

You fixed:

* ✅ skip logic with OR
* ✅ type conversions early
* ✅ sorting using `.items()` correctly
* ✅ tie-break handling via `( -cpu, host )` pattern
* ✅ identified malformed input edge case

To reach **9–10**, the main thing is automatic reflex: **read requirement → choose `>=` vs `>` correctly** and add a **samples==0 guard**.

---

If you want another fresh 5/10 after this, I can do a **“top N by rate”** snippet that forces:

* safe parsing
* dict accumulation
* float division per key
* correct `sorted(key=...)` mental model
  (without repeating the exact same log theme).
